## AI Inference from a video file
This notebook is an example how to use DeGirum PySDK to do AI inference of a video stream from a video file. The annotated video is saved into new file with `_annotated` suffix.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

#### Specify video file name here

In [ ]:
input_filename = "./images/Traffic.mp4" # video file to process

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = mytools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, mytools.get_cloud_zoo_url(), mytools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
import cv2
from pathlib import Path
import IPython.display

In [ ]:
# load object detection model
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("yolo_v5s_coco--512x512_quant_n2x_orca1_1")

# set model parameters
model.overlay_show_probabilities = True
model.overlay_line_width = 1

In [ ]:
orig_path = Path(input_filename)
ann_path = orig_path.with_name(orig_path.stem + "_annotated" + orig_path.suffix)

# AI prediction loop
# Press 'x' or 'q' to stop
with mytools.Display("AI Camera") as display, \
     mytools.open_video_stream(input_filename) as stream, \
     mytools.open_video_writer(str(ann_path), stream.get(cv2.CAP_PROP_FRAME_WIDTH), stream.get(cv2.CAP_PROP_FRAME_HEIGHT)) as writer:
     
    progress = mytools.Progress(int(stream.get(cv2.CAP_PROP_FRAME_COUNT)))
    for res in model.predict_batch(mytools.video_source(stream)):
        img = res.image_overlay
        writer.write(img)
        display.show(img)
        progress.step()


In [ ]:
# display result
IPython.display.Video(ann_path)

In [ ]:
# display original video
IPython.display.Video(orig_path)